In [32]:
import duckdb

In [33]:
duckdb.sql("INSTALL httpfs; LOAD httpfs")

In [39]:
# Connect or Create a database
conn = duckdb.connect("../air_quality.db")

In [41]:
# conn.execute("DROP schema IF EXISTS raw CASCADE")
conn.execute("CREATE schema IF NOT EXISTS raw")

In [42]:
# OpenAQ's S3 bucket is public, 
# however, 
# DuckDB requires access and secret keys, 
# therefore, 
# set to blank

conn.sql("""
SET s3_access_key_id='';
SET s3_secret_access_key='';
SET s3_region='';
""")


In [43]:
# Create a raw table in the database to store the data coming from the S3 bucket (the column names in double quotes are keywords in DuckDB)
conn.execute("""
    CREATE TABLE IF NOT EXISTS raw.air_quality_data (
        location_id BIGINT,
        sensors_id BIGINT,
        "location" VARCHAR,
        "datetime" TIMESTAMP,
        lat DOUBLE,
        lon DOUBLE,
        "parameter" VARCHAR,
        units VARCHAR,
        "value" DOUBLE,
        "month" VARCHAR,
        "year" BIGINT,
        ingestion_datetime TIMESTAMP
    );
""")

In [44]:
# insert the data from OpenAQ's s3 bucket

# s3_bucket = 's3://openaq-data-archive/records/csv.gz/locationid=42400/year=2023/month=01/*.csv.gz'
s3_bucket = 's3://openaq-data-archive/records/csv.gz/locationid=225393/year=2024/month=01/*.csv.gz'

try:
    conn.execute(f"""
        INSERT INTO raw.air_quality_data
        SELECT
                 location_id,
                 sensors_id,
                 "location",
                 "datetime",
                 lat,
                 lon,
                 "parameter",
                 units,
                 "value",
                 "month",
                 "year",
                 current_timestamp AS ingestion_datetime
        FROM read_csv('{s3_bucket}')
    """)
except Exception as e:
    print(f"An error occurred: {e}")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [45]:
# check the content of the database using pandas
import pandas as pd

# Query the database using the existing connection
df = pd.read_sql_query("SELECT COUNT(*) FROM raw.air_quality_data;", con=conn)

print(df)

   count_star()
0          3278


/tmp/ipykernel_213151/4287083592.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query("SELECT COUNT(*) FROM raw.air_quality_data;", con=conn)


In [47]:
# close the connection
conn.close()